In [49]:
import os
import pandas as pd

main_publisher = 'Khan'

script_dir = os.path.dirname(os.path.realpath('__file__'))
data_path = os.path.join(script_dir, '../Data/' + main_publisher + '/data/')
df_concepts = pd.read_csv(data_path + 'concepts_bis.csv', sep = '|')
print(df_concepts.shape)
df_concepts.head()

(15252, 3)


,OER,Concept,PR
0,0,http://en.wikipedia.org/wiki/Antibiotic,0.018078
1,0,http://en.wikipedia.org/wiki/Antimicrobial_res...,0.005757
2,0,http://en.wikipedia.org/wiki/Ape,0.005058
3,0,http://en.wikipedia.org/wiki/Bacteria,0.022978
4,0,http://en.wikipedia.org/wiki/DNA,0.007154


In [50]:
from SPARQLWrapper import SPARQLWrapper, JSON, XML
import time

def get_classes(concept):
    classes = []
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setReturnFormat(JSON)
    request = """
                    SELECT ?class WHERE {
            <http://dbpedia.org/resource/CONCEPT> <http://purl.org/dc/terms/subject> ?class
        }
    """
    request = request.replace('CONCEPT', concept)
    sparql.setQuery(request)
    done = False
    while not done : 
        try:
            ret = sparql.queryAndConvert()

            for r in ret["results"]["bindings"]:
                value = r["class"]["value"]
                if ("/yago/" not in value) and ("/umbel/" not in value) and (".wikidata." not in value):
                    classes.append({
                        "Concept" : concept,
                        "Class" : value
                    })
            done = True
            print(f'Concept {concept} done.')
        except Exception as e:
            print('Runtime Error : ', e)
            print('Sleeping for 10 seconds...')
            time.sleep(10)
            print('Sleep done !')
    return classes

In [51]:
y = [get_classes(x.split('/')[-1]) for x in df_concepts['Concept'].unique()]
y = [item for sublist in y for item in sublist]
df_classes = pd.DataFrame(y)
df_classes.head(20)
print(df_classes.shape)
df_classes.to_csv(data_path + "classes_new.csv", sep = '|')

Concept Antibiotic done.
Concept Antimicrobial_resistance done.
Concept Ape done.
Concept Bacteria done.
Concept DNA done.
Concept Immune_system done.
Concept Industrial_Revolution done.
Concept Moth done.
Concept Natural_selection done.
Concept Peppered_moth done.
Concept Virus done.
Concept Art done.
Concept Binomial_nomenclature done.
Concept Chimpanzee done.
Concept Common_name done.
Concept Eponym done.
Concept Evolution done.
Concept Feeling done.
Concept Gibbon done.
Concept Gorilla done.
Concept Great_cormorant done.
Concept Hominidae done.
Concept Human done.
Concept Justice done.
Concept Object_(philosophy) done.
Concept Ontology done.
Concept Order_(biology) done.
Concept Phenotypic_trait done.
Concept Primate done.
Concept Rights done.
Concept Sense done.
Concept Tail done.
Concept Tao done.
Concept Video done.
Concept %2eso done.
Concept Forgiveness done.
Concept Fractal done.
Concept God done.
Concept Godless_(miniseries) done.
Concept Idea done.
Concept Intelligent_desig